In [1]:
# import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime
from sklearn import preprocessing, linear_model, model_selection

%matplotlib inline

Класс моделей ARIMA недостаточно богат для наших данных: с их помощью, например, никак нельзя учесть взаимосвязи между рядами. Это можно сделать с помощью векторной авторегрессии VARIMA, но её питоновская реализация не позволяет использовать регрессионные признаки. Кроме того, авторегрессионный подход не позволяет учитывать, например, взаимодействия между сезонными компонентами. Вы могли заметить, что форма суточных сезонных профилей в будни и выходные немного разная; явно моделировать этот эффект с помощью ARIMA не получится.

Нам нужна более сложная модель. Давайте займёмся сведением задачи массового прогнозирования рядов к регрессионной постановке! Вам понадобится много признаков. Некоторые из них у вас уже есть — это:
<ol>
<li>идентификатор географической зоны
<li>дата и время
<li>количество поездок в периоды, предшествующие прогнозируемому
<li>синусы, косинусы и тренды, которые вы использовали внутри регрессионной компоненты ARIMA
<li>Кроме того, не спешите выбрасывать построенный вами на прошлой неделе прогнозы — из них может получиться хороший признак для регрессии!
</ol>



Вы можете попробовать разные регрессионный модели, но хорошие результаты, скорее всего, дадут такие, которые будут позволять признакам взаимодействовать друг с другом.

Поскольку прогноз нужен на 6 часов вперёд, проще всего будет построить 6 независимых регрессионных моделей — одна для прогнозирования y^T+1|T, другая для y^T+2|T и т.д.

<ol>Чтобы сдать задание, выполните следующую последовательность действий.
<li>Для каждой из шести задач прогнозирования y^T+i|T,i=1,…,6 сформируйте выборки. Откликом будет yT+i при всевозможных значениях T, а признаки можно использовать следующие:
<ul>
<li>идентификатор географической зоны — категориальный
<li>год, месяц, день месяца, день недели, час — эти признаки можно пробовать брать и категориальными, и непрерывными, можно даже и так, и так (done)
<li>синусы, косинусы и тренды, которые вы использовали внутри регрессионной компоненты ARIMA (done)
<li>сами значения прогнозов ARIMA y^T+i|TARIMA
<li>количество поездок из рассматриваемого района в моменты времени yT,yT−1,…,yT−K (параметр K можно подбирать; попробуйте начать, например, с 6)
<li>количество поездок из рассматриваемого района в моменты времени yT−24,yT−48,…,yT−24∗Kd (параметр Kd можно подбирать; попробуйте начать, например, с 2)
<li>суммарное количество поездок из рассматриваемого района за предшествующие полдня, сутки, неделю, месяц
</ul>
Будьте внимательны при создании признаков — все факторы должны быть рассчитаны без использования информации из будущего: при прогнозировании y^T+i|T,i=1,…,6 вы можете учитывать только значения y до момента времени T включительно.


<li>Выбранными моделями постройте для каждой географической зоны и каждого конца истории от 2016.04.30 23:00 до 2016.05.31 17:00 прогнозы на 6 часов вперёд; посчитайте в ноутбуке ошибку прогноза по следующему функционалу:
Qmay=1R∗739∗6∑r=1R∑T=2016.04.3023:002016.05.3117:00∑i=16y^T|T+ir−yT+ir.
Убедитесь, что ошибка полученных прогнозов, рассчитанная согласно функционалу Q, определённому на прошлой неделе, уменьшилась по сравнению с той, которую вы получили методом индивидуального применения моделей ARIMA. Если этого не произошло, попробуйте улучшить ваши модели.

<li>Итоговыми моделями постройте прогнозы для каждого конца истории от 2016.05.31 23:00 до 2016.06.30 17:00 и запишите все результаты в один файл в формате geoID, histEndDay, histEndHour, step, y. Здесь geoID — идентификатор зоны, histEndDay — день конца истории в формате id,y, где столбец id состоит из склеенных через подчёркивание идентификатора географической зоны, даты конца истории, часа конца истории и номера отсчёта, на который делается предсказание (1-6); столбец y — ваш прогноз.

<li>Загрузите полученный файл на kaggle: https://inclass.kaggle.com/c/yellowtaxi. Добавьте в ноутбук ссылку на сабмишн.

<li>Загрузите ноутбук в форму.

Подгружаем данные

In [2]:
# id нужных регионов
regsDf = pd.read_csv('../crowdRegs.csv',names=['id','regId']);  

# времянные ряды для этих регионов
df = pd.read_pickle('../loadData/crowdRegs3.pcl')
regNames = regsDf.regId.values.astype('str')
df.columns = regNames

Наверное, оптимальный способ - пройтись по всем регионам, сформировать требуемую выборку, а потом - состыковать. 
Вероятно, в процессе работы получится векторизовать это действие.
Пожалуй, имеет смысл сначала для всего фрейма добавить общие для всех колонок признаки (тренд, гармоники, даты, дни недели)

In [3]:
def processDataFrame(df2, Kw = 5, Ka = 2):
    """
    Обрабатываем сразу весь dateFrame и добавляем признаки, общие для всех рядов
    тренд, гармоники, категориальные перемнные
    для дат, дней недели, etc)

    Parameters:
    Kw number of weeks harmonics
    Ka number of annual harmonics
    """

    df2 = df2.assign(linear = (df2.index - datetime.datetime(2014,1,1,0,0,0))/np.timedelta64(1, 'h'))
    
    # час — эти признаки можно пробовать брать и категориальными
    # и непрерывными, можно даже и так, и так

    # добавляем гармонические фичи
    for ind in range(1,Kw+1):
        df2['weekCos'+str(ind)]= np.cos(np.pi*df2.linear*ind/168)
        df2['weekSin'+str(ind)]= np.sin(np.pi*df2.linear*ind/168)
     
    for ind in range(1,Ka+1):
        df2['yearCos'+str(ind)]= np.cos(2*np.pi*df2.linear*ind/8766)        
        df2['yearSin'+str(ind)]= np.sin(2*np.pi*df2.linear*ind/8766)

    # добавляем числовое и категориальные свойства для дней недели
    df2 = df2.assign(dayOfWeek = df2.index.dayofweek)
    lbDays = preprocessing.LabelBinarizer()
    lbDays.fit(list(np.arange(6)))
    DoW = pd.DataFrame(lbDays.transform(df2.index.dayofweek),columns = ['dayOfWeek_'+str(x) for x in np.arange(6)],
                       index = df2.index)      
    df2 = df2.merge(DoW,left_index=True,right_index=True)

    # добавляем dummy variables для месяца
    df2 = df2.assign(month = df2.index.month)
    lbMonths = preprocessing.LabelBinarizer()
    lbMonths.fit(list(np.arange(12)))
    Months = pd.DataFrame(lbMonths.transform(df2.index.month),columns = ['month_'+str(x) for x in np.arange(1,13)],
                          index = df2.index)      
    df2 = df2.merge(Months,left_index=True,right_index=True);

    # добавляем год (вещественный)
    df2 = df2.assign(year = df.index.year)

    # добавляем день месяца (вещественный)
    df2 = df2.assign(day = df.index.day)

    # добавляем час (вещественный и категориальный)
    df2 = df2.assign(hour = df.index.hour)
    lbHours = preprocessing.LabelBinarizer()
    lbHours.fit(list(np.arange(24)))
    Hours = pd.DataFrame(lbHours.transform(df2.index.hour),columns = ['hour_'+str(x) for x in np.arange(24)],
                       index = df2.index)      
    df2 = df2.merge(Hours,left_index=True,right_index=True)
    
    return df2

Теперь делаем индивидуальную обработку для каждого региона
<ol>
<li> добавляем идентификатор географической зоны — категориальный
<li> количество поездок из рассматриваемого района в моменты времени yT,yT−1,…,yT−K (параметр K можно подбирать; попробуйте начать, например, с 6)
<li> количество поездок из рассматриваемого района в моменты времени yT−24,yT−48,…,yT−24∗Kd (параметр Kd можно подбирать; попробуйте начать, например, с 2)
<li>суммарное количество поездок из рассматриваемого района за предшествующие полдня, сутки, неделю, месяц 
2) 
</ol>

In [4]:
def processSeries(df,tReg,Kh = 6, Kp = 2):
    """
    Обработка одного данного ряда 
    parameters:
        df - начальный датафрейм, из которого выберем для обработки один ряд
        tReg - название ряда, который надо обработать
        Kh - количество отслеживаемых прошлых суточных лагов "назад"
        Kp - количество отслеживаемых прошлых периодических лагов (период 24 часа)

    """
 

    tDf = df.loc[:,tReg.split() + commonFeatures].rename(columns={tReg:'y'})

    tDf = tDf.assign(region = tReg)

    for timeLag in np.arange(1,Kh+1):
        name = 'hourLag_'+str(timeLag)
        tDf.loc[:,name] = tDf.y.shift(periods=timeLag)

    for timeLag in np.arange(1,Kp+1):
        name = 'periodicLag_'+str(timeLag)
        tDf.loc[:,name] = tDf.y.shift(periods=timeLag*24)

    tDf.fillna(0,inplace=True)    

    # суммарное количество поездок из рассматриваемого района за предшествующие полдня, сутки, неделю, месяц
    tDf.loc[:,'sum12'] = tDf.y.rolling(window = 12, min_periods = 1).sum()
    tDf.loc[:,'sum24'] = tDf.y.rolling(window = 24, min_periods = 1).sum()
    tDf.loc[:,'sumWeek'] = tDf.y.rolling(window = 168, min_periods = 1).sum()
    tDf.loc[:,'sumMonth'] = tDf.y.rolling(window = 720, min_periods = 1).sum()
    
    #создаём шесть целевые переменных для каждого конца истории
    for targetVar in np.arange(1,7):
        name = 'y'+str(targetVar)
        tDf.loc[:,name] = tDf.y.shift(-targetVar)
    tDf.fillna(0,inplace=True)
    
    return tDf


In [5]:
# общая обработка данных
df2 = processDataFrame(df,Kw = 5, Ka = 5)

In [6]:
commonFeatures =  list(set(df2.columns)-set(df.columns.values))

In [7]:
# общая обработка данных
df = processDataFrame(df,Kw = 5, Ka = 5)

# обработка отдельный рядов
df3 = pd.DataFrame()
for regName in regNames:
    print regName
    df3 = pd.concat([df3, processSeries(df,regName,Kh = 12, Kp = 4)])

1075
1076
1077
1125
1126
1127
1128
1129
1130
1131
1132
1172
1173
1174
1175
1176
1177
1178
1179
1180
1181
1182
1183
1184
1221
1222
1223
1224
1225
1227
1228
1229
1230
1231
1232
1233
1234
1235
1272
1273
1274
1278
1279
1280
1281
1282
1283
1284
1285
1286
1287
1326
1327
1331
1332
1333
1334
1335
1336
1337
1338
1339
1376
1377
1378
1380
1382
1383
1384
1385
1386
1387
1388
1389
1390
1426
1431
1434
1435
1436
1437
1438
1439
1441
1442
1480
1482
1483
1530
1532
1533
1580
1630
1684
1733
1734
1783
2068
2069
2118
2119
2168


Разбейте каждую из шести выборок на три части:
<ul>
<li> Обучающая, на которой будут настраиваться параметры моделей — всё до апреля 2016
<li> Тестовая, на которой вы будете подбирать значения гиперпараметров — май 2016
<li> Итоговая, которая не будет использоваться при настройке моделей вообще — июнь 2016
</ul>

In [8]:
df3 = pd.get_dummies(df3,'regions')
df3.reset_index(inplace=True)

In [9]:
startTrain = '2014-01-01 00:00:00'
endTrain   = '2016-04-30 23:00:00'

startValidation = '2016-05-01 00:00:00'
endValidation   = '2016-05-31 23:00:00'

startTest = '2016-06-01 00:00:00'
endTest   = '2016-06-30 23:00:00'

In [10]:
trainSet = df3.query('index >= @startTrain and index <= @endTrain') 
validationSet = df3.query('index >= @startValidation and index <= @endValidation')
testSet = df3.query('index >= @startTest and index <= @endTest')

Выберите вашу любимую регрессионную модель и настройте её на каждом из шести наборов данных, подбирая гиперпараметры на мае 2016. Желательно, чтобы модель:
<ul>
<li>Допускала попарные взаимодействия между признаками
<li>Была устойчивой к избыточному количеству признаков (например, использовала регуляризаторы)
</ul>

Попробуем обучить Ridge классификатор с L2 регуляризацией.

In [11]:
targetList = ['y1','y2','y3','y4','y5','y6']
paramsAlpha = np.linspace(200,300,5)

In [16]:
bestModels = dict()
for target in targetList:
    print target
    dropCols = [x for x in targetList if x != target ]
    dropCols.append('y')
    dropCols.append('index')
    tmpTrain = trainSet.drop(dropCols,axis = 1)
    tmpTrain.rename(columns={target:'y'},inplace=True)
    
    tmpValidation = validationSet.drop(dropCols,axis = 1)
    tmpValidation.rename(columns={target:'y'},inplace=True)
    
    minErr = np.inf
    bestAlpha = 0
    
    
    for a in paramsAlpha:
        regressor = linear_model.Ridge(alpha= a)
        regressor.fit(tmpTrain.drop('y',axis = 1),tmpTrain.loc[:,'y'])
        prediction = regressor.predict(tmpValidation.drop('y',axis = 1))
        err = np.abs(prediction - tmpValidation.loc[:,'y'])
        err = err.mean()
        print a, err
        if err <minErr:
            minErr = err
            bestAlpha = a
            bestModel = regressor
    
    bestModels.update({target:bestModel})
            
    print 'Smallest error {:2.3f} at a = {:2.3f}'.format(err,a)     

y1
200.0 28.4032538264
225.0 28.3999056262
250.0 28.3967319173
275.0 28.3937040474
300.0 28.3908031744
Smallest error 28.391 at a = 300.000
y2
200.0 43.5291749434
225.0 43.5219953666
250.0 43.5152190627
275.0 43.5088047822
300.0 43.5027023054
Smallest error 43.503 at a = 300.000
y3
200.0 49.7291047296
225.0 49.7199307006
250.0 49.7113413037
275.0 49.7032127339
300.0 49.6955130444
Smallest error 49.696 at a = 300.000
y4
200.0 50.7505021265
225.0 50.7404470377
250.0 50.7310304816
275.0 50.7221545072
300.0 50.7137371473
Smallest error 50.714 at a = 300.000
y5
200.0 49.777043812
225.0 49.7655537082
250.0 49.7548052053
275.0 49.744687815
300.0 49.7351137299
Smallest error 49.735 at a = 300.000
y6
200.0 48.4908573692
225.0 48.4764622848
250.0 48.4630773065
275.0 48.4504868387
300.0 48.4386439819
Smallest error 48.439 at a = 300.000


In [17]:
bestModels

{'y1': Ridge(alpha=300.0, copy_X=True, fit_intercept=True, max_iter=None,
    normalize=False, random_state=None, solver='auto', tol=0.001),
 'y2': Ridge(alpha=300.0, copy_X=True, fit_intercept=True, max_iter=None,
    normalize=False, random_state=None, solver='auto', tol=0.001),
 'y3': Ridge(alpha=300.0, copy_X=True, fit_intercept=True, max_iter=None,
    normalize=False, random_state=None, solver='auto', tol=0.001),
 'y4': Ridge(alpha=300.0, copy_X=True, fit_intercept=True, max_iter=None,
    normalize=False, random_state=None, solver='auto', tol=0.001),
 'y5': Ridge(alpha=300.0, copy_X=True, fit_intercept=True, max_iter=None,
    normalize=False, random_state=None, solver='auto', tol=0.001),
 'y6': Ridge(alpha=300.0, copy_X=True, fit_intercept=True, max_iter=None,
    normalize=False, random_state=None, solver='auto', tol=0.001)}

In [19]:
# now there are three regressors. WE have to use them.
np.save('bestModels.npy',bestModels)

In [13]:
bestModels = np.load('bestModels.npy').item()

In [15]:
testSet.columns.values
dropCols = ['index','y','y1','y2','y3','y4','y5','y6']
tripVal = testSet.get(dropCols)
testSet.drop(dropCols,inplace = True, axis = 1)

D:\Anaconda2\lib\site-packages\ipykernel\__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [16]:
testSet

,weekSin1,weekSin2,month,hour_9,hour_8,year,hour_5,hour_4,hour_7,hour_6,...,regions_1630,regions_1684,regions_1733,regions_1734,regions_1783,regions_2068,regions_2069,regions_2118,regions_2119,regions_2168
21168,-5.095769e-14,-1.019154e-13,6,0,0,2016,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
21169,1.869887e-02,3.739119e-02,6,0,0,2016,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
21170,3.739119e-02,7.473009e-02,6,0,0,2016,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
21171,5.607045e-02,1.119645e-01,6,0,0,2016,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
21172,7.473009e-02,1.490423e-01,6,0,0,2016,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
21173,9.336361e-02,1.859116e-01,6,0,0,2016,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
21174,1.119645e-01,2.225209e-01,6,0,0,2016,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
21175,1.305262e-01,2.588190e-01,6,0,0,2016,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
21176,1.490423e-01,2.947552e-01,6,0,1,2016,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
21177,1.675062e-01,3.302791e-01,6,1,0,2016,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [18]:
bestModels

{'y1': Ridge(alpha=300.0, copy_X=True, fit_intercept=True, max_iter=None,
    normalize=False, random_state=None, solver='auto', tol=0.001),
 'y2': Ridge(alpha=300.0, copy_X=True, fit_intercept=True, max_iter=None,
    normalize=False, random_state=None, solver='auto', tol=0.001),
 'y3': Ridge(alpha=300.0, copy_X=True, fit_intercept=True, max_iter=None,
    normalize=False, random_state=None, solver='auto', tol=0.001),
 'y4': Ridge(alpha=300.0, copy_X=True, fit_intercept=True, max_iter=None,
    normalize=False, random_state=None, solver='auto', tol=0.001),
 'y5': Ridge(alpha=300.0, copy_X=True, fit_intercept=True, max_iter=None,
    normalize=False, random_state=None, solver='auto', tol=0.001),
 'y6': Ridge(alpha=300.0, copy_X=True, fit_intercept=True, max_iter=None,
    normalize=False, random_state=None, solver='auto', tol=0.001)}

In [20]:
model1 = bestModels.get('y1')
model2 = bestModels.get('y2')
model3 = bestModels.get('y3')
model4 = bestModels.get('y4')
model5 = bestModels.get('y5')
model6 = bestModels.get('y6')

In [23]:
model1.predict(testSet.iloc[21168,:])

D:\Anaconda2\lib\site-packages\sklearn\utils\validation.py:395: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)


array([ 161.10568726])